# Political Organizations Activity on Wikipedia aggregated by Party

The parameters in the cell below can be adjusted to explore other political parties and time frames.

### How to explore other political parties?
The ***organization*** parameter can be use to aggregate organizations by their party. The column `subcategory` in this [this other notebook](../political_organizations.ipynb?autorun=true) show the organizations that belong each party.

***Alternatively***, you can direcly use the [organizations API](http://mediamonitoring.gesis.org/api/organizations/swagger/), or access it with the [SMM Wrapper](https://pypi.org/project/smm-wrapper/).

## A. Set Up parameters

In [ ]:
# Parameters: 
organization = 'afd'
from_date = '2017-09-01'
to_date = '2018-12-31'
aggregation = 'week'

## B. Using APIs
### B.1 Using the SMM Organization API

In [ ]:
library("httr")
library("jsonlite")
library('dplyr', warn.conflicts = FALSE)

# prepare urls
base <- "http://mediamonitoring.gesis.org/api/organizations/"
url_all <- paste(base, "all/", sep="")
url_chobs <- paste(base, "wikipedia/chobs/organizations/", sep="")

# prepare parameters
params = list(
    from_date=from_date,
    to_date=to_date,
    aggregate_by=aggregation
)

# use the api to get the organizations and filter the parties
orgs <- as.data.frame(fromJSON(content(GET(url_all), "text", encoding="UTF-8"), flatten = TRUE))
party_df <- orgs[orgs$subcategory == organization & !is.null(orgs$wp_ids),]

# query the Social Media Monitoring API for Wikipedia changes 
chobs <- data.frame()
for (organization_id in party_df$organization_id) {
    json_chobs <- fromJSON(content(GET(paste(url_chobs, organization_id, "/?", sep=""),query=params), "text", encoding="UTF-8"), flatten = TRUE)

    # concatenate
    if (length(json_chobs$values) != 0) {
        chobs <- rbind(chobs, as.data.frame(json_chobs))
    }    
}

# group by day, week, or month, and then merge
chobs <- summarise(group_by(chobs, labels), values = sum(values))


# replace missing with 0
chobs[is.na(chobs)] <- 0

### B.2 Using the Wikiwho API

In [ ]:
#prepare urls
base_ww <- "https://www.wikiwho.net/de/edit_persistence/v1.0.0-beta/page/"
                   
#prepare parameters
params = list(
    start=from_date,
    end=to_date 
)

# query the Wikiwho API for Wikipedia actions (edit_persistance)
wikiwho <- data.frame()
for (wp_ids in party_df$wp_ids) {
    for (wp_id in wp_ids) {
        ### B.3 Using the Wikimedia API
        json_wikiwho <- fromJSON(content(GET(paste(base_ww, wp_id ,"/?",sep=""), query=params), "text", encoding="UTF-8"))
        wikiwho <- rbind(wikiwho, json_wikiwho$editions)
    }
}

# calculate undos
wikiwho$undos <- wikiwho$dels + wikiwho$reins

# calculate conflict score
wikiwho <- summarise(group_by(wikiwho, year_month), undos = sum(undos),
                     conflict = sum(conflict), elegibles= sum(elegibles))

# format the dates
wikiwho$conflict_score <- wikiwho$conflict / wikiwho$elegibles

# replace missing with 0
wikiwho[is.na(wikiwho)] <- 0

### B.3 Using the Wikimedia API

In [ ]:
library('forcats')

# prepare the urls
pre <- "https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/de.wikipedia.org/all-access/user/"
post <- paste("/daily/", gsub("-", "", from_date),"/", gsub("-", "", to_date), sep="")

# get the data throught the Wikimedia API
appended_views = list()
for (wp_titles in party_df$wp_titles) {
    for (wp_title in wp_titles) {       
        x <- tryCatch({
            view = as.data.frame(fromJSON(content(GET(
                paste(pre, URLencode(wp_title, reserved=TRUE), post, sep="")), "text", 
                encoding="UTF-8"), flatten = TRUE))
            appended_views <- append(appended_views, list(view))
        }, error = function(e) e )
    }
}
views  <- dplyr::bind_rows(appended_views)

# prepare the dates in the right format
views$items.timestamp<-as.POSIXct(strptime(views[["items.timestamp"]], 
                                           format='%Y%m%d%H'),tz = 'GMT')

# # daily, weekly or monthly aggregation of the data
views<-summarise(group_by(views, time = forcats::fct_explicit_na(cut(items.timestamp, aggregation))), 
                 items.views = sum(items.views))

# replace missing with 0
views[is.na(views)] <- 0

## C. Plotting
### C.1 Plot Wikipedia Activity

In [ ]:
library("ggplot2")
options(repr.plot.width=8, repr.plot.height=3)

# merge data so that the x scales are equal
wiki_data = merge( views, chobs, by.x='time', by.y='labels', all.x = TRUE)

#plotting wikipedia changes
ggplot(data = wiki_data, mapping = aes(as.Date(time))) +
  geom_line(aes(y = values), na.rm=TRUE) + 
  labs(title = "Wikipedia Activity", y = "Changes") +
  theme(axis.text.x = element_blank(), axis.title.x = element_blank()) +
  scale_x_date(date_breaks = "1 month")

#plotting wikipedia views
ggplot(data = wiki_data, mapping = aes(as.Date(time))) +
  geom_line(aes(y = items.views)) +   
  labs(y = "Views") +
  theme(axis.text.x = element_text(angle = 60, hjust = 0.5, vjust = 0.5),
        axis.title.x = element_blank()) +
  scale_x_date(date_breaks = "1 month")

### C.2 Plot Wikipedia Disagreement

In [ ]:
# plotting wikipedia undo actions
ggplot(data = wikiwho, mapping = aes(as.Date(year_month))) +
  geom_line(aes(y = undos)) +
  labs(title = "Wikipedia Disagreements", y = "Undos") +
  theme(axis.text.x = element_blank(), axis.title.x = element_blank()) +
  scale_x_date(date_breaks = "1 month")

# plotting conflict
ggplot(data = wikiwho, mapping = aes(as.Date(year_month))) +
  geom_line(aes(y = conflict_score)) +
  labs(y = "Conflict") +
  theme(axis.text.x = element_text(angle = 60, hjust = 0.5, vjust = 0.5),
        axis.title.x = element_blank()) +
  scale_x_date(date_breaks = "1 month")